In [ ]:
%pip install jaal

In [ ]:
import pandas as pd


from aperturedb.CommonLibrary import (
    execute_query,
    create_connector
)
from aperturedb.Utils import Utils


In [ ]:
client=create_connector()
utils = Utils(client)
utils.summary()


In [ ]:
# This is the list of movie ids that we will use to visualize the graph
# They are the movies where Tom Hanks has been involved.
movie_ids = [
                    64685,
                    9800,
                    10193,
                    5255,
                    140823,
                    5516,
                    13448,
                    20763,
                    591,
                    862,
                    83542,
                    863,
                    8358,
                    9591,
                    59861,
                    4147,
                    594,
                    6538,
                    35,
                    920,
                    857,
                    568,
                    9489,
                    497,
                    13508,
                    2619,
                    13,
                    109424,
                    640,
                    2280,
                    11287,
                    296098]

In [ ]:
q = [
    {
        "FindEntity": {
            "with_class": "MOVIE",
            "constraints": {
                "movie_id": ["in",  movie_ids]
            },
            "_ref": 1,
            "results": {
                "all_properties": True,
            },
        }
    },
    {
        "FindEntity": {
            "with_class": "PROFESSIONAL",
            "is_connected_to": {
                "ref": 1,
            },
            "results": {
                "all_properties": True,
                "group_by_source": True,
            },
        }
    },
    {
        "FindEntity": {
            "with_class": "GENRE",
            "is_connected_to": {
                "ref": 1,
            },
            "results": {
                "all_properties": True,
                "group_by_source": True,
            }
        }
    }
]

result, response, b = execute_query(client, q)
types = ["PROFESSIONAL", "GENRE"]
nodes = []
nodeset = set()
edges = set()
for e in response[0]['FindEntity']['entities']:
    nodes.append({
        "id": e['_uniqueid'],
        "label": e['title'],
        "budget": e['budget'],
        "popularity": e['popularity'],
        "type": "MOVIE"
    })
    nodeset.add(e['_uniqueid'])
for src in response[0]['FindEntity']['entities']:
    src_id = src['_uniqueid']
    for i, sr in enumerate(response[1:]):
        for e in sr['FindEntity']['entities'][src_id]:
            if e['_uniqueid'] not in nodeset:
                nodes.append({
                    "id": e['_uniqueid'],
                    "label": e['name'],
                    "gender": e['gender'] if 'gender' in e else None,
                    "budget": 10000,
                    # "type": "PROFESSIONAL" if 'job' in e else "GENRE"
                    "type": e['label']
                })
                nodeset.add(e['_uniqueid'])
            edges.add((src_id, e['_uniqueid']))

nodes_df = pd.DataFrame(nodes)
nodes_df

In [ ]:
cq = []
i = 0
for src, dst in edges:
    cq += [
        {
            "FindEntity": {
                "_ref": 1 + i,
                "constraints": {
                    "_uniqueid": ["==", src],
                },
                "results": {
                    "all_properties": True,
                },
            }
        },
        {
            "FindEntity": {
                "_ref": 2 + i,
                "constraints": {
                    "_uniqueid": ["==", dst],
                },
                "results": {
                    "all_properties": True,
                },
            }
        },
        {
            "FindConnection": {
                "src": 1 + i,
                "dst": 2 + i,
                # "with_class": "CAST",
                "results": {
                    "all_properties": True,
                },
            }
        },
    ]
    i += 2

connections = set()

result, response, b = execute_query(client, cq)
for i in range(0, len(response), 3):
    src = response[i]["FindEntity"]["entities"][0]["_uniqueid"]
    dst = response[i + 1]["FindEntity"]["entities"][0]["_uniqueid"]
    if "connections" not in response[i + 2]["FindConnection"]:
        continue
    for c in response[i + 2]["FindConnection"]["connections"]:
        if "character" in c:
            connections.add((src, dst, c["character"]))
            break
        else:
            if "job" in c:
                id = f'{c["job"]}_{c["department"]}'
                connections.add((src, dst, id))
                break
            else:
                id = f'{c["_uniqueid"]}'
                connections.add((src, dst, id))
                break



edge_df = pd.json_normalize(
    [
        {"from": s, "to": d, "label": id, "id": f"{s}_{d}_{id}"}
        for s, d, id in connections
    ]
)
edge_df

In [ ]:
from jaal import Jaal
Jaal(edge_df, nodes_df).plot()